In [1]:
!pip install langchain
!pip install boto3
!pip install langchain-community
!pip install langchain-aws
!pip install gpt4all
!pip install langchain-qdrant

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 457.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.

In [2]:
!python -c "from langchain_community.embeddings.gpt4all import GPT4AllEmbeddings; embedding_model = GPT4AllEmbeddings(model_name='all-MiniLM-L6-v2.gguf2.f16.gguf')"

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in GPT4AllEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
Downloading: 100% 45.9M/45.9M [00:01<00:00, 35.4MiB/s]
Verifying: 100% 45.9M/45.9M [00:00<00:00, 479MiB/s]
Failed to load libllamamodel-mainline-cuda.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory
Failed to load libllamamodel-mainline-cuda-avxonly.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [3]:
# !pip install aws configure
# !pip install awscli

# !aws configure

In [4]:
import boto3
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.gpt4all import GPT4AllEmbeddings
from langchain_aws.llms.bedrock import BedrockLLM
from langchain.chains import RetrievalQA
from langchain_qdrant import QdrantVectorStore

from google.colab import userdata

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in GPT4AllEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [5]:
def get_bedrock_runtime(region_name, *args, **kwargs):
    return boto3.client(service_name='bedrock-runtime', region_name=region_name, *args, **kwargs)


def get_langchain_bedrock_llm(model_id, client, *args, **kwargs):
    return BedrockLLM(model_id=model_id, client=client, *args, **kwargs)

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=900,
    chunk_overlap=20,
    length_function=len
)

In [7]:
embed_model_name = 'all-MiniLM-L6-v2.gguf2.f16.gguf'
qdrant_url = userdata.get('APOLOGETICS_QDRANT_URL')
qdrant_api_key = userdata.get('LANGSMITH_API_KEY')
llm_name = 'mistral.mixtral-8x7b-instruct-v0:1'

In [8]:
aws_access_token = userdata.get('AWS_STEPHENHO_SERVERLESS_ACC_ACCESS_TOKEN')
aws_secret_key = userdata.get('AWS_STEPHENHO_SERVERLESS_ACC_SECRET_ACCESS_KEY')

config = {
      "max_tokens": 4096,
      "temperature": 0.7,
      "top_p": 0.8
    }

bedrock_runtime = get_bedrock_runtime('us-east-1', aws_access_key_id=aws_access_token, aws_secret_access_key=aws_secret_key)
llm = get_langchain_bedrock_llm(llm_name, bedrock_runtime)

In [9]:
embedding_model = GPT4AllEmbeddings(model_name=embed_model_name)

In [10]:
qdrant = QdrantVectorStore.from_existing_collection(
    embedding=embedding_model,
    url=qdrant_url,
    api_key=qdrant_api_key,
    collection_name='apologetics',
    prefer_grpc=True
)

In [11]:
retriever = qdrant.as_retriever()

In [12]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever, return_source_documents=True)

In [13]:
question = 'What is the ontological argument for the existence of God?'

In [14]:
result_json = qa.invoke({'query': question})

In [15]:
result_json

{'query': 'What is the ontological argument for the existence of God?',
 'result': " The ontological argument for the existence of God is an argument from the concept or idea of God to God's existence. It comes in two forms: one from the idea of a Perfect Being and the other from the idea of a Necessary Being. The Perfect Being form argues that the mere concept of God as an absolutely perfect being demands that he exist, because existence is a perfection. The Necessary Being form argues that existence is logically necessary to the concept of a Necessary Being, and therefore a Necessary Being (God) necessarily exists.",
 'source_documents': [Document(metadata={'page': 490, 'source': '/content/drive/MyDrive/Apologetics Corpus/Baker Encyclopedia of Christian Apologetics (Baker Reference Library) (Norman L. Geisler).pdf', '_id': '0bc3567d-a92b-4389-8f27-fadbd383837d', '_collection_name': 'apologetics'}, page_content='ontological argument (though not the first to call it this) was Anselm (1

In [16]:
import json

In [17]:
def convert_langchaindoc_to_dict(doc):
  return {
      'page_content': doc.page_content,
      'metadata': doc.metadata
  }

In [18]:
result_dict = {
    'query': result_json['query'],
    'answer': result_json['result'],
    'source_documents': [convert_langchaindoc_to_dict(doc) for doc in result_json['source_documents']]
}

In [19]:
json.dumps(result_dict)

'{"query": "What is the ontological argument for the existence of God?", "answer": " The ontological argument for the existence of God is an argument from the concept or idea of God to God\'s existence. It comes in two forms: one from the idea of a Perfect Being and the other from the idea of a Necessary Being. The Perfect Being form argues that the mere concept of God as an absolutely perfect being demands that he exist, because existence is a perfection. The Necessary Being form argues that existence is logically necessary to the concept of a Necessary Being, and therefore a Necessary Being (God) necessarily exists.", "source_documents": [{"page_content": "ontological argument (though not the first to call it this) was Anselm (1033\\u20131109). In its simplest form it\\nargues from the idea of God to the existence  of God. There are two forms of the argument: one from\\nthe idea of a Perfect Being and the other from the idea of a Necessary Being.\\nThe Perfect Being.  According to th